### 90803 Data Cleaning and Question Definition
# Merge all data (ACS, crime, covid) together

**Team 14**

Colton Lapp & Chi-Shiun Tsai

In [1]:
#import modules 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from census import Census
from us import states
import numpy as np
import os
import seaborn as sns
import us
import re

# 1. Read in all cleaned datsets

In [2]:
base_dir = os.getcwd().split('/')
base_dir = '/'.join(base_dir[0:-1])

ACS_data = pd.read_csv( base_dir + '/data/data_cleaned/ACS_5yr_data.csv')
covid_data = pd.read_csv( base_dir + '/data/data_cleaned/covid_cities.csv')
crime_data = pd.read_csv( base_dir + '/data/data_cleaned/crime_cities.csv')

print("Shape of city data: ", ACS_data.shape)
print("Shape of covid data: ", covid_data.shape)
print("Shape of crime data: ", crime_data.shape)

Shape of city data:  (19729, 61)
Shape of covid data:  (29357, 5)
Shape of crime data:  (4240, 7)


/var/folders/xm/xqtx9_vj3398yls687yyxg180000gn/T/ipykernel_98321/2641066915.py:4: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  ACS_data = pd.read_csv( base_dir + '/data/data_cleaned/ACS_5yr_data.csv')


## 2. Merge all data together

In [3]:
#Merge all to ACS data
data_merged = ACS_data.merge(covid_data.drop('NAMELSAD', axis=1), on='GEOID', how='left')
data_merged = data_merged.merge(crime_data.drop(['STATEFP', 'PLACEFP', 'NAMELSAD', 'City'], axis=1), on='GEOID', how='left')

In [4]:
data_merged.info()
data_merged.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19758 entries, 0 to 19757
Data columns (total 66 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Name                                                       19758 non-null  object 
 1   Population 2020                                            19758 non-null  int64  
 2   Median Household Income                                    19758 non-null  float64
 3   Median Home Value                                          19758 non-null  float64
 4   Median Gross Rent                                          19758 non-null  float64
 5   Renter Household                                           19758 non-null  float64
 6   Total Housing Units                                        19758 non-null  float64
 7   Race: White Alone                                          19758 non-null  float64
 8   Hispan

,Name,Population 2020,Median Household Income,Median Home Value,Median Gross Rent,Renter Household,Total Housing Units,Race: White Alone,Hispanic or Latino,Population Age 25 or Older: Bachelor`s Degree,...,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,year,cases_per_100K,deaths_per_100K,State Name,TotalOffenses
0,"New York city, New York",8379552,67046.0,635200.0,1489.0,2143429.0,42.002186,41.330205,28.925997,16.000569,...,A,777982727,434642000,40.663468,-73.938697,2020.0,5913.616647,220.640533,NaN,NaN
1,"Los Angeles city, California",3973278,65290.0,670700.0,1523.0,884176.0,38.099297,48.930480,48.066307,16.225771,...,A,1214591312,87468461,34.019394,-118.410825,2020.0,6178.068445,67.066531,NaN,NaN
2,"Chicago city, Illinois",2699347,62097.0,267600.0,1154.0,590848.0,45.110392,47.731581,28.628813,16.758127,...,A,588881854,17621529,41.837551,-87.681844,2020.0,7499.869841,133.847542,NaN,NaN
3,"Houston city, Texas",2313238,53600.0,186800.0,1086.0,499608.0,42.481318,51.539271,44.471948,13.576078,...,A,1658093679,81248080,29.785743,-95.388806,2020.0,4991.616106,57.677759,Texas,227553.0
4,"Phoenix city, Arizona",1658422,60914.0,250800.0,1100.0,258077.0,37.805637,68.233176,42.647046,11.954557,...,A,1340766401,3236736,33.572154,-112.090132,2020.0,7307.164034,116.139965,NaN,NaN


# 3. Subset correct predictor variables
There are some leftover variables from the shapefile merges we don't actually need, so drop them.

In [5]:
# Subset target variables
dta1 = data_merged.iloc[:, 0:43]
dta1.head(1)


,Name,Population 2020,Median Household Income,Median Home Value,Median Gross Rent,Renter Household,Total Housing Units,Race: White Alone,Hispanic or Latino,Population Age 25 or Older: Bachelor`s Degree,...,Population 35-64: No Health Insurance Coverage,Households with No Internet Acces,Households Receiving Food Stamps/SNAP,Households Below the Poverty Leve,Gross Rent 50% + of Household Income,Owner Households by Year Moved In: 2017 or Later,Total Vacant Housing Units,Housing Units Built in 2014 or Later,state,place
0,"New York city, New York",8379552,67046.0,635200.0,1489.0,2143429.0,42.002186,41.330205,28.925997,16.000569,...,3.427486,5.030627,7.244218,6.55683,6.763679,0.208245,3.913145,0.719466,36,51000


In [6]:
# Subset target variables
dta2 = data_merged.iloc[:, 43:]
dta2 = dta2[['GEOID', 'Population 2019', 'Pop Chng %', 'ALAND', 'AWATER', 'cases_per_100K', 'deaths_per_100K']]
dta2.columns = ['GEOID', 'Population 2019', 'Pop Chng %', 'land_area', 'water_area', 'cases_per_100K', 'deaths_per_100K']
dta2.head(1)

,GEOID,Population 2019,Pop Chng %,land_area,water_area,cases_per_100K,deaths_per_100K
0,3651000,8419316,-0.472295,777982727,434642000,5913.616647,220.640533


In [7]:
dta = pd.concat([dta1, dta2], axis=1)
dta.head(5)

,Name,Population 2020,Median Household Income,Median Home Value,Median Gross Rent,Renter Household,Total Housing Units,Race: White Alone,Hispanic or Latino,Population Age 25 or Older: Bachelor`s Degree,...,Housing Units Built in 2014 or Later,state,place,GEOID,Population 2019,Pop Chng %,land_area,water_area,cases_per_100K,deaths_per_100K
0,"New York city, New York",8379552,67046.0,635200.0,1489.0,2143429.0,42.002186,41.330205,28.925997,16.000569,...,0.719466,36,51000,3651000,8419316,-0.472295,777982727,434642000,5913.616647,220.640533
1,"Los Angeles city, California",3973278,65290.0,670700.0,1523.0,884176.0,38.099297,48.930480,48.066307,16.225771,...,0.887101,6,44000,644000,3966936,0.159871,1214591312,87468461,6178.068445,67.066531
2,"Chicago city, Illinois",2699347,62097.0,267600.0,1154.0,590848.0,45.110392,47.731581,28.628813,16.758127,...,0.868655,17,14000,1714000,2709534,-0.375969,588881854,17621529,7499.869841,133.847542
3,"Houston city, Texas",2313238,53600.0,186800.0,1086.0,499608.0,42.481318,51.539271,44.471948,13.576078,...,2.628869,48,35000,4835000,2310432,0.121449,1658093679,81248080,4991.616106,57.677759
4,"Phoenix city, Arizona",1658422,60914.0,250800.0,1100.0,258077.0,37.805637,68.233176,42.647046,11.954557,...,1.114433,4,55000,455000,1633017,1.555709,1340766401,3236736,7307.164034,116.139965


### Combine some ACS columns to make single variable
These variables need to be summed to be interpreted correctly because each individual variable only represents part of the total count.

In [8]:
# Combine columns
dta['Employed in Professional/Scientific/Technical Services'] = dta['Employed in Professional/Scientific/Technical Services'] + dta['Employed in Professional/Scientific/Technical Services 2']
dta['Employed in Accommodation/Food Services'] = dta['Employed in Accommodation/Food Services'] + dta['Employed in Accommodation/Food Services 2']
dta['Employed in Health Care/Social Assistance'] = dta['Employed in Health Care/Social Assistance'] + dta['Employed in Health Care/Social Assistance 2']
dta['Employed in Arts/Entertainment/Recreation'] = dta['Employed in Arts/Entertainment/Recreation'] + dta['Employed in Arts/Entertainment/Recreation 2']

dta.drop(['Employed in Professional/Scientific/Technical Services 2', 'Employed in Accommodation/Food Services 2',
           'Employed in Health Care/Social Assistance 2', 'Employed in Arts/Entertainment/Recreation 2'], axis=1, inplace=True)
dta.head(2)


,Name,Population 2020,Median Household Income,Median Home Value,Median Gross Rent,Renter Household,Total Housing Units,Race: White Alone,Hispanic or Latino,Population Age 25 or Older: Bachelor`s Degree,...,Housing Units Built in 2014 or Later,state,place,GEOID,Population 2019,Pop Chng %,land_area,water_area,cases_per_100K,deaths_per_100K
0,"New York city, New York",8379552,67046.0,635200.0,1489.0,2143429.0,42.002186,41.330205,28.925997,16.000569,...,0.719466,36,51000,3651000,8419316,-0.472295,777982727,434642000,5913.616647,220.640533
1,"Los Angeles city, California",3973278,65290.0,670700.0,1523.0,884176.0,38.099297,48.930480,48.066307,16.225771,...,0.887101,6,44000,644000,3966936,0.159871,1214591312,87468461,6178.068445,67.066531


In [9]:
# Check for missing values
dta.isnull().sum(axis=0)

Name                                                           0
Population 2020                                                0
Median Household Income                                        0
Median Home Value                                              0
Median Gross Rent                                              0
Renter Household                                               0
Total Housing Units                                            0
Race: White Alone                                              0
Hispanic or Latino                                             0
Population Age 25 or Older: Bachelor`s Degree                  0
Population Age 25 or Older: Doctorate Degree                   0
Pop 5-17 Speak only English at Home                            0
Population Age 15 or Older Never Married                       0
Population Age 15 or Older Never Married 2                     0
Population Age 65 or more                                      0
Pop 65 or more Living in 

# 4. Drop rows with missing observations
Some rows have missing values for COVID-19 data. Because the impact of COVID-19 is one of our main questions, we can't use these rows in our model, so we drop them.

In [10]:
data_complete = dta.dropna()
print("Number of observations: ", data_complete.shape)
data_complete.head(10)

Number of observations:  (19495, 46)


,Name,Population 2020,Median Household Income,Median Home Value,Median Gross Rent,Renter Household,Total Housing Units,Race: White Alone,Hispanic or Latino,Population Age 25 or Older: Bachelor`s Degree,...,Housing Units Built in 2014 or Later,state,place,GEOID,Population 2019,Pop Chng %,land_area,water_area,cases_per_100K,deaths_per_100K
0,"New York city, New York",8379552,67046.0,635200.0,1489.0,2143429.0,42.002186,41.330205,28.925997,16.000569,...,0.719466,36,51000,3651000,8419316,-0.472295,777982727,434642000,5913.616647,220.640533
1,"Los Angeles city, California",3973278,65290.0,670700.0,1523.0,884176.0,38.099297,48.930480,48.066307,16.225771,...,0.887101,6,44000,644000,3966936,0.159871,1214591312,87468461,6178.068445,67.066531
2,"Chicago city, Illinois",2699347,62097.0,267600.0,1154.0,590848.0,45.110392,47.731581,28.628813,16.758127,...,0.868655,17,14000,1714000,2709534,-0.375969,588881854,17621529,7499.869841,133.847542
3,"Houston city, Texas",2313238,53600.0,186800.0,1086.0,499608.0,42.481318,51.539271,44.471948,13.576078,...,2.628869,48,35000,4835000,2310432,0.121449,1658093679,81248080,4991.616106,57.677759
4,"Phoenix city, Arizona",1658422,60914.0,250800.0,1100.0,258077.0,37.805637,68.233176,42.647046,11.954557,...,1.114433,4,55000,455000,1633017,1.555709,1340766401,3236736,7307.164034,116.139965
5,"Philadelphia city, Pennsylvania",1581531,49127.0,171600.0,1084.0,289341.0,43.573031,39.330687,15.055791,12.279304,...,0.868842,42,60000,4260000,1579075,0.155534,347781835,21806065,5098.950713,149.751446
6,"San Antonio city, Texas",1529133,53420.0,156700.0,1025.0,235647.0,36.304102,71.900613,64.734526,10.707702,...,1.824498,48,65000,4865000,1508083,1.395812,1256436954,14515625,5135.625695,100.480272
7,"San Diego city, California",1414545,83454.0,629500.0,1770.0,268694.0,38.806189,61.953278,30.064014,19.137320,...,1.182218,6,66000,666000,1409573,0.352731,844018494,120537242,4682.894250,46.149634
8,"Dallas city, Texas",1338846,54747.0,208700.0,1111.0,307837.0,43.469301,57.749136,41.483860,14.159806,...,2.263442,48,19000,4819000,1330612,0.618813,879911395,115783461,6006.289751,59.002442
9,"San Jose city, California",1029409,117324.0,925800.0,2232.0,140841.0,32.844962,37.490055,30.991375,18.088243,...,1.097814,6,68000,668000,1027690,0.167268,460521077,8129848,3368.690908,38.217801


In [11]:
# Check for missing values again to make sure that the data is complete
data_complete.isnull().sum(axis=0)

Name                                                         0
Population 2020                                              0
Median Household Income                                      0
Median Home Value                                            0
Median Gross Rent                                            0
Renter Household                                             0
Total Housing Units                                          0
Race: White Alone                                            0
Hispanic or Latino                                           0
Population Age 25 or Older: Bachelor`s Degree                0
Population Age 25 or Older: Doctorate Degree                 0
Pop 5-17 Speak only English at Home                          0
Population Age 15 or Older Never Married                     0
Population Age 15 or Older Never Married 2                   0
Population Age 65 or more                                    0
Pop 65 or more Living in Nonfamily Households          

# 5. Save as csv

In [12]:
data_complete.to_csv('../data/data_cleaned/data_all.csv', index=False)

### References

1. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
2. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html